In [1]:
import parse

In [32]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2024, day=24)

def parses(data):
    
    signals, gates = data.strip().split('\n\n')
    signals = dict([parse.parse('{}: {:d}', line).fixed for line in signals.split('\n')])
    gates = [parse.parse('{} {} {} -> {}', line).fixed for line in gates.split('\n')] 
    gates = {c: (op, a, b) for a, op, b, c in gates}
    return signals, gates

data = parses(puzzle.input_data)

In [12]:
sample = parses("""x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02""")

In [22]:
OPS = {
    'AND': lambda x, y: x & y,
    'OR': lambda x, y: x | y,
    'XOR': lambda x, y: x ^ y,
}

In [376]:
def solve_a(data):

    signals, gates = data
    signals, gates = signals.copy(), gates.copy()

    changed = True
    while changed:
        changed = False

        for c, (op, a, b) in list(gates.items()):
            if a in signals and b in signals:
                signals[c] = OPS[op](signals[a], signals[b])
                gates.pop(c)
                changed = True

    num = 0
    for k, v in signals.items():
        if k.startswith('z') and v == 1:
            i = int(k[1:])
            num |= (1 << i)
    return num

In [377]:
solve_a(sample)

4

In [378]:
sample2 = parses("""x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj""")

In [379]:
solve_a(sample2)

2024

In [33]:
solve_a(data)

58740594706150

In [388]:
def solve_b(data):
    _, gates = data
    wrong = []
    highest_z = sorted([g for g in gates if g[0] == 'z'])[-1]
    for c, (op, a, b) in sorted(gates.items()):
        
        valid = True
        if c[0] == 'z' and c != highest_z:
            i = int(c[1:])
            valid &= op == 'XOR'
            if c == 'jfw': print(1, valid)
            
        if op == 'XOR':
            # Either output must be a z, or must take in both x and y
            valid &= (c[0] == 'z') or (['x','y'] == sorted([a[0], b[0]]))
            
        if op == "AND":
            valid  &= (
                ['x00', 'y00'] == sorted([a, b]) or
                all(op2 == "OR" for _, (op2, a2, b2) in gates.items() if c in [a2, b2])
            )
            
        if op in ('OR', 'XOR'):
            valid &= all(op2 in ['AND', 'XOR'] for c2, (op2, a2, b2) in gates.items() if c in [a2,b2])
            
        if not valid:
            wrong.append(c)
            
    return ','.join(sorted(wrong))

In [389]:
solve_b(data)

'cvh,dbb,hbk,kvn,tfn,z14,z18,z23'

In [390]:
cvh,dbb,hbk,kvn,tfn,z14,z18,z23

NameError: name 'cvh' is not defined

In [385]:
gates['jfw']

('AND', 'x00', 'y00')

In [383]:
gates['z45']

('OR', 'cjf', 'pfk')

In [346]:
sol

{'cvh', 'dbb', 'hbk', 'kvn', 'tfn', 'z14', 'z18', 'z23'}

In [357]:
gates['cvh']

('AND', 'y34', 'x34')

In [362]:
gates['tfn']

('XOR', 'x34', 'y34')

In [35]:
len(data[1])

222

In [227]:
def from_signals(signals, var: str):
    num = 0
    for k, v in signals.items():
        if k.startswith(var) and v == 1:
            i = int(k[1:])
            num |= (1 << i)
    return num

In [228]:
def as_signals(num, name):
    i = 0
    signals = {}
    for _ in range(45):
        signals[f'{name}{i:02d}'] = num & 1
        i += 1
        num >>= 1
    return signals

In [231]:
def simulate(x, y, gates):
    gates = gates.copy()
    signals = as_signals(x, 'x') | as_signals(y, 'y')
    changed = True
    while changed:
        changed = False

        for c, (op, a, b) in list(gates.items()):
            if a in signals and b in signals:
                signals[c] = OPS[op](signals[a], signals[b])
                gates.pop(c)
                changed = True
    
    z = from_signals(signals, 'z')
    return z==x+y, signals


In [240]:
signals, gates = data

s2 = simulate(10,10, gates)[1]

In [241]:
dictdiff(s1, s2)

['ntt', 'sqm', 'ndd', 'z02', 'npm', 'z04']

In [242]:
def dictdiff(d1, d2):
    return [k for k in d1|d2 if d1.get(k)!=d2.get(k) and k[0] not in ('xy')]

In [321]:
ref_signals = simulate(0,0, gates)[1]

correct = set()
maybe_incorrect = set()

for i in range(43):
    ok, signals = simulate(3<<i,1<<i, gates)
    changed = set(dictdiff(signals, ref_signals))
    if ok:
        correct |= changed
    else:
        maybe_incorrect |= changed
    print(maybe_incorrect-correct)

set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
{'z15', 'rmg', 'hbk', 'cbr', 'scs', 'bfn'}
{'z15', 'rmg', 'sjr', 'hbk', 'cbr', 'scs', 'dfb', 'bfn', 'z14', 'wjj'}
{'z15', 'rmg', 'bkb', 'sjr', 'hbk', 'cbr', 'scs', 'dfb', 'bfn', 'tck', 'z14', 'wjj'}
{'z15', 'rmg', 'bkb', 'sjr', 'hbk', 'cbr', 'scs', 'dfb', 'bfn', 'tck', 'z14', 'wjj'}
{'sjr', 'cjv', 'z15', 'dfb', 'bfn', 'tck', 'kvn', 'rmg', 'z19', 'hbk', 'cbr', 'prt', 'z14', 'fgr', 'bkb', 'scs', 'cjb', 'kbh', 'wjj'}
{'rmg', 'z15', 'bkb', 'sjr', 'hbk', 'cbr', 'scs', 'prt', 'kbh', 'dfb', 'bfn', 'cjv', 'tck', 'kvn', 'z14', 'wjj'}
{'rmg', 'z15', 'bkb', 'sjr', 'hbk', 'cbr', 'srm', 'scs', 'prt', 'kbh', 'dfb', 'bfn', 'cjv', 'tck', 'kvn', 'z14', 'z18', 'wjj'}
{'rmg', 'z15', 'bkb', 'sjr', 'hbk', 'cbr', 'srm', 'scs', 'prt', 'kbh', 'dfb', 'bfn', 'cjv', 'tck', 'kvn', 'z14', 'z18', 'wjj'}
{'rmg', 'z15', 'bkb', 'sjr', 'hbk', 'cbr', 'srm', 'scs', 'prt', 'kbh', 'dfb', 'bfn', 'cjv', 'tck', 'kvn', 'z14', 'z18', 'wjj'}
{'wjj', 'sjr', '

In [322]:
from math import comb
comb(33,4)

40920

In [317]:
incorrect = maybe_incorrect - correct

In [318]:
len(incorrect)

33

In [319]:
sol = set(['tfn', 'cvh', 'z18', 'kvn', 'z14', 'hbk', 'z23', 'dbb'])

In [320]:
sol - incorrect

set()

In [36]:
def as_signals(num, name):
    i = 0
    signals = {}
    while num:
        signals[f'{name}{i}'] = num & 1
        i += 1
        num >>= 1
    return signals

In [38]:
as_signals(5,'x')

{'x0': 1, 'x1': 0, 'x2': 1}

In [39]:
def simulate(data, x, y):

    signals, gates = data
    signals = {k: 0 for k in signals} | as_signals(x, 'x') | as_signals(y, 'y')
    gates = gates.copy()
    
    changed = True
    while changed:
        changed = False

        for c, (op, a, b) in list(gates.items()):
            if a in signals and b in signals:
                signals[c] = OPS[op](signals[a], signals[b])
                gates.pop(c)
                changed = True

    num = 0
    for k, v in signals.items():
        if k.startswith('z') and v == 1:
            i = int(k[1:])
            num |= (1 << i)
    return num

In [169]:
def translate(data, swaps):
    signals, gates = data
    signals, gates = signals.copy(), gates.copy()
    
    for a, b in swaps:
        gates[a], gates[b] = gates.pop(b), gates.pop(a)
    
    names = {}
    def rename(orig, new):
        if any(c in '0123456789' for c in orig):
            return
        names[new] = orig
        if orig not in gates: 
            return
        gates[new] = gates.pop(orig)
        for c, (op, a, b) in gates.items():
            a = a if a != orig else new
            b = b if b != orig else new
            gates[c] = (op, a, b)
    
    for c, (op, a, b) in list(gates.items()):
        if op == 'XOR' and a[0] == 'x' and b[0] == 'y' and a[1:] == b[1:]:
            rename(c, f's{a[1:]}')
        if op == 'AND' and a[0] == 'x' and b[0] == 'y' and a[1:] == b[1:]:
            rename(c, f'a{a[1:]}')
            
        if op == 'XOR' and b[0] == 'x' and a[0] == 'y' and a[1:] == b[1:]:
            rename(c, f's{a[1:]}')
        if op == 'AND' and b[0] == 'x' and a[0] == 'y' and a[1:] == b[1:]:
            rename(c, f'a{a[1:]}')
            
    for _ in range(45):
        for c, (op, a, b) in list(gates.items()):
            if op == 'XOR' and c.startswith('z'):
                if a.startswith('s') and c[1:] == a[1:]:
                    rename(b, f'c{a[1:]}')
                if b.startswith('s') and c[1:] == b[1:]:
                    rename(a, f'c{b[1:]}')

            if op == 'AND' and a[0] == 'c' and b[0] == 's' and a[1:] == b[1:]:
                rename(c, f'b{a[1:]}')
#                 i = 1 + int(a[1:])
#                 rename(c, f'b{i:02d}')
            if op == 'AND' and a[0] == 's' and b[0] == 'c' and a[1:] == b[1:]:
                rename(c, f'b{a[1:]}')
            
            if op == 'OR' and a[0] == 'a' and b[0] == 'b' and a[1:] == b[1:]:
                i = 1 + int(a[1:])
                rename(c, f'c{i:02d}')
            if op == 'OR' and a[0] == 'b' and b[0] == 'a' and a[1:] == b[1:]:
                i = 1 + int(a[1:])
                rename(c, f'c{i:02d}')
                
    return gates, names

In [182]:
gates, names = translate(data, [
    ('tfn', 'cvh'), 
    ('z18', 'kvn'),
    ('z14', 'hbk'), 
    ('z23', 'dbb')
])

In [186]:
swaps = [
    ('tfn', 'cvh'), 
    ('z18', 'kvn'),
    ('z14', 'hbk'), 
    ('z23', 'dbb')
]
','.join(sorted([i for pair in swaps for i in pair]))

'cvh,dbb,hbk,kvn,tfn,z14,z18,z23'

In [183]:
have = set()
for c, (op, a, b) in gates.items():
    a, b = sorted([a,b])
    rule = f'{c} = {a} {op} {b}'
    have.add(rule)
        
want = set(['c01 = x00 AND y00', 'z45 = a44 OR b44', 'z00 = x00 XOR y00', 'z01 = a00 XOR s01' ])
for i in range(45):
    want.add(f'a{i:02d} = x{i:02d} AND y{i:02d}')
    want.add(f's{i:02d} = x{i:02d} XOR y{i:02d}')
    want.add(f'z{i:02d} = c{i:02d} XOR s{i:02d}')
    want.add(f'b{i:02d} = c{i:02d} AND s{i:02d}')
    want.add(f'c{i+1:02d} = a{i:02d} OR b{i:02d}')

In [184]:
sorted(have-want)

['c02 = a01 OR spq', 'spq = a00 AND s01']

'hbk'

In [157]:
[r for r in have if 'b33' in r]

['b33 = c33 AND s33', 'c34 = a33 OR b33']

In [120]:
sorted(gates.items())

[('a01', ('AND', 'x01', 'y01')),
 ('a02', ('AND', 'y02', 'x02')),
 ('a03', ('AND', 'y03', 'x03')),
 ('a04', ('AND', 'x04', 'y04')),
 ('a05', ('AND', 'y05', 'x05')),
 ('a06', ('AND', 'x06', 'y06')),
 ('a07', ('AND', 'x07', 'y07')),
 ('a08', ('AND', 'x08', 'y08')),
 ('a09', ('AND', 'x09', 'y09')),
 ('a10', ('AND', 'x10', 'y10')),
 ('a11', ('AND', 'y11', 'x11')),
 ('a12', ('AND', 'y12', 'x12')),
 ('a13', ('AND', 'y13', 'x13')),
 ('a14', ('AND', 'y14', 'x14')),
 ('a15', ('AND', 'y15', 'x15')),
 ('a16', ('AND', 'y16', 'x16')),
 ('a17', ('AND', 'x17', 'y17')),
 ('a18', ('AND', 'y18', 'x18')),
 ('a19', ('AND', 'x19', 'y19')),
 ('a20', ('AND', 'y20', 'x20')),
 ('a21', ('AND', 'y21', 'x21')),
 ('a22', ('AND', 'y22', 'x22')),
 ('a23', ('AND', 'x23', 'y23')),
 ('a24', ('AND', 'y24', 'x24')),
 ('a25', ('AND', 'x25', 'y25')),
 ('a26', ('AND', 'x26', 'y26')),
 ('a27', ('AND', 'y27', 'x27')),
 ('a28', ('AND', 'y28', 'x28')),
 ('a29', ('AND', 'y29', 'x29')),
 ('a30', ('AND', 'x30', 'y30')),
 ('a31', (

In [ ]:
 ('bfn', ('OR', 'a13', 'b14')),
    
 ('dbb', ('XOR', 'dvw', 's23')),
 ('dvw', ('OR', 'b23', 'a22')),
    
 ('ffb', ('AND', 'fgr', 's18')),

('fgr', ('OR', 'b18', 'a17')),
 ('kvn', ('XOR', 's18', 'fgr')),
    
 ('mqf', ('OR', 'a33', 'b34')),

 ('z45', ('OR', 'b45', 'a44'))]


In [ ]:
new_gates = {}
for c, (op, a, b) in gates.items():
    i = 

In [46]:
from functools import cache

In [63]:
def all_ancestors(data):
    signals, gates = data
    outputs = {z: None for z in gates if z.startswith('z')}
    @cache
    def ancestors(signal):
        if signal in signals:
            return [signal]
        _, a, b = gates[signal]
        pa = ancestors(a) if a not in outputs else [a]
        pb = ancestors(b) if b not in outputs else [b]
        return sorted(set(pa + pb))
    return {s: ancestors(s) for s in outputs}

In [89]:
# all_ancestors(sample2)

In [65]:
anc = all_ancestors(data)

In [42]:
len(data[1])

222

In [43]:
222*(222-1)/2

24531.0

In [44]:
from math import comb

In [45]:
comb(24531, 4)

15084938352824340

In [41]:
simulate(sample, 0, 0)

0

In [26]:
num

4

In [ ]:
def solve_a(data):
    pass

In [ ]:
solve_a(sample)

In [ ]:
solve_a(data)

In [ ]:
def solve_b(data):
    pass

In [ ]:
solve_b(sample)

In [ ]:
solve_b(data)